# AIWIN-互联网舆情企业风险事件的识别和预警

赛题链接：http://ailab.aiwin.org.cn/competitions/48


## 读取数据

In [1]:
import pandas as pd # 读取文件pandas 结构化数据
import jieba # 中文分词
import numpy as np # 矩阵/线性运算

# 绘图函数
%pylab inline
import seaborn as sns

# 不要显示警告
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
company_name = pd.read_excel('2_公司实体汇总_20210414_A1.xlsx', names=['name'])

In [3]:
company_name.head()

,name
0,武汉卡伊娜化妆品有限公司
1,北京泰禾锦绣置业有限公司
2,长城新盛信托有限责任公司
3,招商银行股份有限公司
4,中国长城资产管理股份有限公司


In [7]:
train_data = pd.read_excel('3_训练集汇总_20210414_A1.xlsx')
train_data.drop(0, inplace=True, axis=0)
# train_data = train_data[['NEWS_TITLE', 'COMPANY_NM', 'LABEL']]

In [12]:
test_data = pd.read_excel('测试集-1.xlsx')
test_data = test_data.iloc[:22288]

In [27]:
test_data

,NEWS_BASICINFO_SID,NEWS_TITLE,ABSTRACT,CONTENT,AUTHOR,SRC_URL,SOURCE_TYPE,PUBLISH_SITE,FIRST_WEB,CHANNEL,NOTICE_DT
0,267188181.0,海能达在美“中招”摩托罗拉，陪审团裁决赔53亿元,NaN,"<article class=""article"" id=""mp-editor"">\n ...",知产力,http://www.sohu.com/a/373747099_221481,01,知产力,搜狐新闻,搜狐新闻,2020-02-17 18:16:00
1,267188183.0,博天环境收监管函：募资补充流动资金到期无法归还 涉3.1亿元,NaN,"<div id=""content"" class=""J-article-content art...",点点新闻,http://mini.eastday.com/a/200108164603613.html,09,中国科技新闻,中国科技新闻,点点新闻,2020-01-08 16:46:00
2,267188188.0,房企生存艰难群像：卖项目求生 中小房企生存难,NaN,<p>广州，房价近四年连续上涨的一线城市，今年却出现了多家破产房企；还有多家中小房企或项目公...,乐居网,https://feng.ifeng.com/c/7uGQGsUm52Q,01,乐居网,大风号,大风号,2020-02-22 07:08:35
3,267188190.0,康盛股份：实际控制人将变更为解直锟,NaN,<p><span>17:16</span>【康盛股份：实际控制人将变更为解直锟】康盛股份晚间...,第一财经,https://www.yicai.com/brief/100408733.html,01,第一财经,第一财经,正在,2019-11-19 17:16:00
4,267188194.0,衣服丑，价格高！“中国版Zara”关店2400家，坑惨投资者！,NaN,"<article class=""article"" id=""mp-editor"">\n ...",财经锐眼,http://www.sohu.com/a/332422701_545836,01,财经锐眼,搜狐新闻,搜狐新闻,2019-08-08 19:52:00
...,...,...,...,...,...,...,...,...,...,...,...
22283,310002394.0,云商集团与您互惠互利共创财富,/,云商集团与您互惠互利共创财富,/,/,/,/,/,/,/
22284,310002395.0,山东能源集团：“众创”热情高 攒足发展后劲,/,山东能源集团：“众创”热情高 攒足发展后劲,/,/,/,/,/,/,/
22285,310002396.0,2016中国华能集团公司博士后科研工作站博士后招收公告,/,2016中国华能集团公司博士后科研工作站博士后招收公告,/,/,/,/,/,/,/
22286,310002397.0,3月24日文通集团钾盐价格动态,/,3月24日文通集团钾盐价格动态,/,/,/,/,/,/,/


## 数据处理

In [13]:
def replace_postfix(s):
    for x in '有限责任公司 股份有限公司 控股股份公司 总公司 有限公司 投资合伙企业 子公司 公司 集团'.split(' '):
        if s[-len(x):] == x:    
            return s[:-len(x)], x
        
    return s, None

company_name['name2'] = company_name['name'].apply(replace_postfix)
company_name['name_short'] = company_name['name2'].apply(lambda x: x[0])
company_name['name_postfix'] = company_name['name2'].apply(lambda x: x[1])

In [14]:
company_name.head()

,name,name2,name_short,name_postfix
0,武汉卡伊娜化妆品有限公司,"(武汉卡伊娜化妆品, 有限公司)",武汉卡伊娜化妆品,有限公司
1,北京泰禾锦绣置业有限公司,"(北京泰禾锦绣置业, 有限公司)",北京泰禾锦绣置业,有限公司
2,长城新盛信托有限责任公司,"(长城新盛信托, 有限责任公司)",长城新盛信托,有限责任公司
3,招商银行股份有限公司,"(招商银行, 股份有限公司)",招商银行,股份有限公司
4,中国长城资产管理股份有限公司,"(中国长城资产管理, 股份有限公司)",中国长城资产管理,股份有限公司


In [15]:
company_name[company_name['name_postfix'].isnull()]

,name,name2,name_short,name_postfix
87,瑞华会计师事务所（特殊普通合伙）,"(瑞华会计师事务所（特殊普通合伙）, None)",瑞华会计师事务所（特殊普通合伙）,None
118,中国人民财产保险股份有限公司涟水支公司高沟营销服务部,"(中国人民财产保险股份有限公司涟水支公司高沟营销服务部, None)",中国人民财产保险股份有限公司涟水支公司高沟营销服务部,None
138,中国工商银行股份有限公司克拉玛依石油分行,"(中国工商银行股份有限公司克拉玛依石油分行, None)",中国工商银行股份有限公司克拉玛依石油分行,None
139,中国工商银行股份有限公司襄阳分行,"(中国工商银行股份有限公司襄阳分行, None)",中国工商银行股份有限公司襄阳分行,None
216,内黄县农村信用合作联社,"(内黄县农村信用合作联社, None)",内黄县农村信用合作联社,None
...,...,...,...,...
15923,东莞银行股份有限公司合肥分行,"(东莞银行股份有限公司合肥分行, None)",东莞银行股份有限公司合肥分行,None
15924,东吴期货有限公司泉州营业部,"(东吴期货有限公司泉州营业部, None)",东吴期货有限公司泉州营业部,None
15926,大信会计师事务所（特殊普通合伙）,"(大信会计师事务所（特殊普通合伙）, None)",大信会计师事务所（特殊普通合伙）,None
15934,衡水银行股份有限公司站前支行,"(衡水银行股份有限公司站前支行, None)",衡水银行股份有限公司站前支行,None


In [16]:
train_data = train_data[['NEWS_TITLE', 'CONTENT','COMPANY_NM', 'LABEL']]

## 构建模型：风险标签识别

In [17]:
from sklearn.preprocessing import LabelEncoder

# 数据集打散
# 标签不是均匀分布的
train_data = train_data.sample(frac = 1.0)

# 训练集做一个编码
lbl = LabelEncoder().fit(train_data['LABEL'])
train_data['LABEL'] = lbl.transform(train_data['LABEL'])

In [19]:
import jieba

def strcut(s):
    seg_list = jieba.cut(s)
    return ' '.join(list(seg_list))

# 新闻标题
train_title = train_data['NEWS_TITLE'].apply(strcut)
test_title = test_data['NEWS_TITLE'].apply(strcut)

### TFIDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TFIDF / FASTTEXT / WORD2VEC / BERT
# TFIDF -> TF * IDF
# 词袋编码 countervector
# IDF 单词在所有文档出现次数的频率的倒数

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidf = TfidfVectorizer(ngram_range=(1,1))
train_title_ttidf = tfidf.fit_transform(train_title)

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeClassifier

# 定义分类器
clf = RidgeClassifier()

In [25]:
from sklearn.model_selection import train_test_split

# 数据集进行划分（holdout）：按照比例进行划分
# 留出法（按照比例）、kfold、采样
# 训练集、 验证集

# train_data['NEWS_TITLE'] ：原始文本（分词前）
# train_title_ttidf： 分词后并提取TFIDF
# train_data['LABEL']： 分类标签
# stratify：划分数据集的依据（按照标签进行划分）
# test_size：测试集占比
tr_x, val_x, tr_tfidf, val_tfidf, tr_y, val_y = train_test_split(
    train_data['NEWS_TITLE'], train_title_ttidf, train_data['LABEL'],
    stratify = train_data['LABEL'],
    test_size=0.1
)

In [26]:
clf.fit(tr_tfidf, tr_y)
clf.score(val_tfidf, val_y)

0.8708297690333618

## 构建模型：公司主体识别

### 规则匹配

In [24]:
for row in train_data.iloc[:10].iterrows():
    # company_name 已知的企业名字
    match1 = company_name[company_name['name'].apply(lambda x: x in row[1].NEWS_TITLE)]
    if match1.shape[0] > 0:
        match1.loc[:, 'name_len'] = match1['name'].apply(len)
        match1 = match1.sort_values(by='name_len')
        match1 = match1.iloc[-1]['name']
    else:
        match1 = ''
    
    match2 = company_name[company_name['name_short'].apply(lambda x: x in row[1].NEWS_TITLE)]
    if match2.shape[0] > 0 and match1 == '':
        match2.loc[:, 'name_len'] = match2['name_short'].apply(len)
        match2 = match2.sort_values(by='name_len')
        match2 = match2.iloc[-1]['name']
    else:
        match2 = ''
    
    print('标题：', row[1].NEWS_TITLE)
    print('主体标签：', row[1]['COMPANY_NM'])
    print('主体识别结果：', match1 + match2)
    print('')
    
    # break

标题： 银保监会对“原油宝”事件作出处罚，中国银行：认罚、反思！
主体标签： 中国银行股份有限公司
主体识别结果： 中国银行股份有限公司

标题： 德州银行烟台2宗违法遭罚85万 3分支机构行长遭警告
主体标签： 德州银行股份有限公司烟台分行
主体识别结果： 德州银行股份有限公司

标题： 2020年，负债累累的轮胎企业！
主体标签： 山东富宇蓝石轮胎有限公司
主体识别结果： 

标题： 资金管控不严被挪用，浦发银行徐州分行、平安银行南京分行被罚
主体标签： 上海浦东发展银行股份有限公司徐州分行
主体识别结果： 平安银行股份有限公司

标题： 银行代销乱象频出！没销售资格却在卖产品、借贷搭售多套路，这7家银行被“点名”
主体标签： 江苏银行股份有限公司
主体识别结果： 

标题： 财务不真实、虚构中介业务套取手续费  人保财险莆田市多家分支机构合计被罚109.3万元
主体标签： 中国人民财产保险股份有限公司莆田市城厢支公司城南营销服务部
主体识别结果： 

标题： 四川蓝光发展股份有限公司公告（系列）
主体标签： /
主体识别结果： 四川蓝光发展股份有限公司

标题： 被证监会立案调查，德威新材离氢能界“第二个宁德时代”渐行渐远
主体标签： 江苏德威新材料股份有限公司
主体识别结果： 

标题： 2019年国有企业丽水市农业投资发展有限公司公开招聘工作人员公告
主体标签： /
主体识别结果： 

标题： 淮北矿业集团财务有限公司朱仙庄煤矿突水事故已致6人遇难
主体标签： 淮北矿业集团财务有限公司
主体识别结果： 淮北矿业集团财务有限公司



In [28]:
test_data['LABEL'] = lbl.inverse_transform(
    clf.predict(tfidf.transform(test_title))
)

In [32]:
def search_result(row):
    print(row)
    # company_name 已知的企业名字
    match1 = company_name[company_name['name'].apply(lambda x: x in row[1].NEWS_TITLE)]
    if match1.shape[0] > 0:
        match1.loc[:, 'name_len'] = match1['name'].apply(len)
        match1 = match1.sort_values(by='name_len')
        match1 = match1.iloc[-1]['name']
    else:
        match1 = ''
    
    match2 = company_name[company_name['name_short'].apply(lambda x: x in row[1].NEWS_TITLE)]
    if match2.shape[0] > 0 and match1 == '':
        match2.loc[:, 'name_len'] = match2['name_short'].apply(len)
        match2 = match2.sort_values(by='name_len')
        match2 = match2.iloc[-1]['name']
    else:
        match2 = ''
    
    if (match1 + match2) == '':
        return str(int(row[1].NEWS_BASICINFO_SID)) + ',/,' + str(row[1].LABEL)
    else:
        return str(int(row[1].NEWS_BASICINFO_SID)) + ','+ match1+match2 + ','+ str(row[1].LABEL)
    # break

In [33]:
from joblib import Parallel, delayed

In [46]:
test_result = Parallel(n_jobs=5)(delayed(search_result)(row) 
                   for row in test_data.iloc[:5000].iterrows())

In [43]:
import codecs
with codecs.open('result.csv', 'w', encoding='utf-8-sig') as up:
    for x in test_result:
        up.write(x+'\n')

In [44]:
!zip  T1_finlay.zip result.csv

updating: result.csv (deflated 83%)


In [45]:
test_data.shape

(22288, 12)